In [3]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "abc123"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))
rescue_types = ['Water Rescue', 'Mountain or Wildernesss Rescue', 'Disaster or Individual Tracking', 'Reset']

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))
app.layout = html.Div([
    html.Center(children=[
        html.Div(id='hidden-div', style={'display':'none'}),
        html.B(html.H1('Austin Animal Shelter')),
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '200px'}),
        html.B(html.H4('CS-340 Module 7 Project - Ryan Blough'))
    ]),
    html.Hr(),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    dcc.RadioItems(
        id = 'rescue-filter',
        options=[{'label': i, 'value': i} for i in rescue_types],
        value='Reset',
        labelStyle={'display': 'inline-block'}
    ),
    
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable = True,
        row_selectable = "single",
        selected_rows = [0],
        filter_action = "native",
        sort_action = "native",
        page_action = "native",
        page_current = 0,
        page_size = 10,
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='graph2-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('rescue-filter', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'Reset':
        dff = df
    elif filter_type == 'Water Rescue':
        dff = df[df.breed.isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])
                & (df.sex_upon_outcome == 'Intact Female')
                & ((df.age_upon_outcome_in_weeks >= 26) & (df.age_upon_outcome_in_weeks <= 156))]
    elif filter_type == 'Mountain or Wilderness Rescue':
        dff = df[df.breed.isin(['German Shepard', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiller'])
                & (df.sex_upon_outcome == 'Intact Male')
                & ((df.age_upon_outcome_in_weeks >= 26) & (df.age_upon_outcome_in_weeks <= 156))]
    elif filter_type == 'Disaster or Individual Tracking':
        dff = df[df.breed.isin(['Doverman Pinscher', 'German Shepard', 'Golden Retriever', 'Bloodhound', 'Rottweiller'])
                & (df.sex_upon_outcome == 'Intact Male')
                & ((df.age_upon_outcome_in_weeks >= 20) & (df.age_upon_outcome_in_weeks <= 300))]
    
    return dff.to_dict('records')
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return html.Div("No data")
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names = 'breed', title = 'Animal Breed')
    return [
        dcc.Graph(figue = fig)
    ]

@app.callback(
    Output('graph2-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs_2(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.histogram(dff, x = "sex_upon_outcome", title = "Sex Upon Outcome")
    return [
        dcc.Graph(figure=fig)
    ]
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    # Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width': '1000px', 'height': '500px'},
            center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates fo the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            [dl.Marker(position = [row['location_lat'],row['location_long']],
                children = [
                dl.Tooltip(row['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(row['name'])
            ])         
        ])for index, row in dff.iterrows()]
    ])
]
app.run_server(debug=True)


Dash app running on http://127.0.0.1:27980/
